In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import time
import pandas as pd

In [22]:
title_list=[]
#베스트 도전 페이지 파싱을 통해 조회순으로 정렬한 6페이지까지의 베스트 도전 작품 제목 가져오기
for i in range(1, 7):
    URL='https://comic.naver.com/genre/bestChallenge.nhn?m=main&order=ViewCount&page={}'.format(i)
    html=requests.get(URL).text
    soup=BeautifulSoup(html,'html.parser')

    title=soup.select('.challengeTitle > a')

    for x in range(len(title)):
        t=title[x].get_text().strip()
        title_list.append(t)

In [15]:
title_num = []
for i in range(1, 145):
    title_num.append(i)

In [16]:
webtoon_df = pd.DataFrame()
webtoon_df['rank'] = title_num
webtoon_df['title'] = title_list

In [24]:
artist_list = []
#베스트 도전 페이지 파싱을 통해 조회순으로 정렬한 6페이지까지의 베스트 도전 작품 작가이름 가져오기
for i in range(1, 7):
    URL='https://comic.naver.com/genre/bestChallenge.nhn?m=main&order=ViewCount&page={}'.format(i)
    html=requests.get(URL).text
    soup=BeautifulSoup(html,'html.parser')

    artist=soup.select('.challengeInfo a.user')

    for x in range(len(title)):
        t=artist[x].get_text().strip()
        artist_list.append(t)

webtoon_df['artist'] = artist_list

In [28]:
summary_list = []
#베스트 도전 페이지 파싱을 통해 조회순으로 정렬한 6페이지까지의 베스트 도전 작품 summary 가져오기
for i in range(1, 7):
    URL='https://comic.naver.com/genre/bestChallenge.nhn?m=main&order=ViewCount&page={}'.format(i)
    html=requests.get(URL).text
    soup=BeautifulSoup(html,'html.parser')

    summary=soup.select('div.summary')

    for x in range(len(title)):
        t=summary[x].get_text().strip()
        summary_list.append(t)

webtoon_df['summary'] = summary_list

In [30]:
rating_list = []
#베스트 도전 페이지 파싱을 통해 조회순으로 정렬한 6페이지까지의 베스트 도전 작품 평균 평점 가져오기
for i in range(1, 7):
    URL='https://comic.naver.com/genre/bestChallenge.nhn?m=main&order=ViewCount&page={}'.format(i)
    html=requests.get(URL).text
    soup=BeautifulSoup(html,'html.parser')

    rating=soup.select('.rating_type strong')

    for x in range(len(title)):
        t=rating[x].get_text().strip()
        rating_list.append(t)

webtoon_df['rating'] = rating_list

In [32]:
webtoon_df

,rank,title,artist,summary,rating
0,1,여우자매,rkddusl999,구미호 이야기,9.90
1,2,대학원 탈출일지,ari9402,대학원에서 살아남기,9.90
2,3,과학고 생존일지,zupam,과학고에 간 문과생,9.80
3,4,오사와 오폐수들,oreo_hunter,그냥 오사의 일상,9.90
4,5,여기서 한잔할래?,softred2008,수달의 혼술툰,9.70
...,...,...,...,...,...
139,140,행복을 찾다.,dkfqka543,아마 순정만화,9.85
140,141,국밥소녀,isw339,국밥에 미친 소녀,8.55
141,142,이래봬도2학년,dreamfully,초등학교 이야기!,9.72
142,143,뭉가툰,tjdgns6497,뭉가의 일상툰,9.81


In [38]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import time
import pandas as pd

# selenium으로 베스트 도전 1페이지 불러오기 
URL='https://comic.naver.com/genre/bestChallenge.nhn?m=main&order=ViewCount'
driver=webdriver.Chrome('chromedriver.exe')
driver.get(URL)

time.sleep(1)

id_list=[] ; nick_list=[] ; chat_list=[] ; score_list=[]
idlst=[] ; nicklst=[] ; chatlst=[] ; DataFrame_id={} ; DataFrame_chat={}

for i in range(0, 24):
    
    id_list.clear()
    nick_list.clear()
    chat_list.clear()
    
    time.sleep(1)
    
    #전체 웹툰 목록 중 월요일 첫 번째 웹툰으로 페이지 이동
    page=driver.find_elements_by_class_name('challengeTitle')
    page[i].click()
    
    time.sleep(0.5)

    #이동한 페이지 주소 읽고, 파싱하기
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    #최신 별점 평균 점수 수집 (최대 10화 분량)
    score = soup.find_all('strong')
    scorelist=[] ; ii=9
    while score[ii].text[0].isnumeric()==True:
        scorelist.append(float(score[ii].text))
        ii +=1
    score_list.append(sum(scorelist)/len(scorelist))
    
    time.sleep(0.5)

    length=driver.find_elements_by_class_name('title')
    if(len(length)>5):
        length=length[:5]
        #각 회차를 돌며 댓글 작성자 아이디 수집
    for j in range(len(length)):
        #해당 페이지의 회차 모두 가져오기
        titlenum=driver.find_elements_by_class_name('title')
        
        time.sleep(0.5)
        
        webnum=[y.text for y in titlenum]
        enterToon = driver.find_elements_by_partial_link_text(webnum[j])
        
        time.sleep(0.5)
        
        enterToon[0].click()
        
        #현재 댓글 부분 파싱이 안되는 설정이므로 src의 댓글창 페이지로 직접 이동
        #현재 URL주소 복사
        html = driver.current_url
        currentPage=html[html.find('?'):]
        
        #댓글창 페이지 URL
        URL2='https://comic.naver.com/comment/comment.nhn' + currentPage
        print('Title:',title_list[i],i,' : ',URL2)
        
        time.sleep(0.4)
        
        #댓글창 페이지 열기
        driver2 = webdriver.Chrome('chromedriver.exe')
        driver2.get(URL2)
        
        time.sleep(8)
        
        while True:
            try:
                #수집 편의를 위해 클린봇 댓글 필터링 기능 해제
                driver2.find_element_by_class_name('u_cbox_cleanbot_setbutton').click()
                time.sleep(0.4)
                driver2.find_element_by_class_name('u_cbox_layer_cleanbot_checkbox').click()
                time.sleep(0.4)
                driver2.find_element_by_class_name('u_cbox_layer_cleanbot_extrabutton').click()
                time.sleep(0.4)
                break
                
            except:
                driver2.close()
                driver2 = webdriver.Chrome('chromedriver.exe')
                driver2.get(URL2)
                print('driver2 ERROR')
                time.sleep(20)
                continue
        
        #베스트 댓글 아이디 목록 수집
        idbest = driver2.find_elements_by_class_name('u_cbox_id') ;jj=0 
        
        time.sleep(0.1)
        
        nickbest = driver2.find_elements_by_class_name('u_cbox_nick')
        
        time.sleep(0.4)
        
        chatbest = driver2.find_elements_by_class_name('u_cbox_contents') 
        
        time.sleep(1)
        
        while jj<len(idbest):
            id_list.append(idbest[jj].text)
            nick_list.append(nickbest[jj].text)
            chat_list.append(chatbest[jj].text)
            jj +=1
            
        time.sleep(0.3)
        
        #전체 댓글 보기 클릭 & 아이디 목록 수집
        driver2.find_element_by_partial_link_text('전체댓글').click()
        
        time.sleep(0.3)
        
        #최신 댓글부터 가장 나중의 댓글까지 창 이동하며 아이디 목록 수집
        num=0 ; start=0 ; error_back=0
        while True:
            try:
                while start<=0:
                    
                    time.sleep(0.4)
            
                    page=driver2.find_elements_by_class_name('u_cbox_num_page')
                    print('now page: ',int(page[num].text),' index: ',num)
            
                    time.sleep(0.1)
            
                    idd=driver2.find_elements_by_class_name('u_cbox_id')
            
                    time.sleep(0.1)
                
                    nickk=driver2.find_elements_by_class_name('u_cbox_nick')

                    time.sleep(0.1)
            
                    chat=driver2.find_elements_by_class_name('u_cbox_contents')
    
                    time.sleep(0.1)
            
                    while len(idd)>0:
                        id_list.append(nickk[0].text+idd[0].text)
                        #nick_list.append(nickk[0].text)
                        chat_list.append(chat[0].text)
                
                        idd.pop(0)
                        nickk.pop(0)
                        chat.pop(0)
                
                    time.sleep(0.1)
            
                    #페이지가 10의 배수이면 다음 페이지 클릭, 동일 과정 거침
                    try:
                        a=driver2.find_elements_by_class_name('u_cbox_next') #*****CLEAR v
                        if(int(page[num].text) % 10 == 0 and a[0].text != ''):
                            driver2.find_element_by_class_name('u_cbox_next').click()
                    
                            time.sleep(0.1)
                    
                            page=driver2.find_elements_by_class_name('u_cbox_num_page')
                    
                            time.sleep(0.1)
                    
                            idd=driver2.find_elements_by_class_name('u_cbox_id')
                    
                            time.sleep(0.1)
                        
                            nickk=driver2.find_elements_by_class_name('u_cbox_nick')
                            
                            time.sleep(0.1)
                    
                            chat=driver2.find_elements_by_class_name('u_cbox_contents')
                            a.clear()
                    
                            time.sleep(0.1)
            
                            while len(idd)>0:
                                id_list.append(nickk[0].text+idd[0].text)
                                #nick_list.append(nickk[0].text)
                                chat_list.append(chat[0].text)
                        
                                idd.pop(0)
                                nickk.pop(0)
                                chat.pop(0)
                            num=0
                            print('now page: ',int(page[num].text),' index: ',num)
            
                    except IndexError:
                        break
                
                    num +=1
            
                    time.sleep(0.4)
            
                    try:
                        page[num].click()
                
                    except IndexError:
                        break
            
                    page.clear()
   
                    idd.clear()
    
                    time.sleep(0.1)
        
                #댓글 페이지 닫기
                error_back=0
                driver2.close()
                break
        
            except:
                print('ERROR')
                error_back +=1
                if(error_back>10):
                    page[num].click()
                continue
                
        time.sleep(0.4)
        
        #페이지 뒤로 가기, 다시 만화 목록으로 
        driver.back()
        
        time.sleep(1)
        
        titlenum.clear()
        
        #time.sleep(0.5)
        
        enterToon.clear()
        
        #time.sleep(0.5)
        
        idbest.clear()
        nickbest.clear()
        chatbest.clear()
    
    #전체 정식연재 만화 창으로 이동
    time.sleep(1)
    print(title_list[i],"end")
    idlst.append(id_list.copy())
    nicklst.append(nick_list.copy())
    chatlst.append(chat_list.copy())
    
    driver.back()
    
    time.sleep(0.5)
    
    page.clear()
    
    time.sleep(0.5)
    length.clear()

Title: 여우자매 0  :  https://comic.naver.com/comment/comment.nhn?titleId=726786&no=78
now page:  1  index:  0
now page:  2  index:  1
now page:  3  index:  2
now page:  4  index:  3
now page:  5  index:  4
now page:  6  index:  5
now page:  7  index:  6
now page:  8  index:  7
now page:  9  index:  8
now page:  10  index:  9
now page:  11  index:  0
now page:  12  index:  1
now page:  13  index:  2
now page:  14  index:  3
now page:  15  index:  4
now page:  16  index:  5
now page:  17  index:  6
now page:  18  index:  7
now page:  19  index:  8
now page:  20  index:  9
now page:  21  index:  0
now page:  22  index:  1
now page:  23  index:  2
now page:  24  index:  3
now page:  25  index:  4
now page:  26  index:  5
now page:  27  index:  6
now page:  28  index:  7
now page:  29  index:  8
now page:  30  index:  9
now page:  31  index:  0
now page:  32  index:  1
now page:  33  index:  2
now page:  34  index:  3
now page:  35  index:  4
now page:  36  index:  5
now page:  37  index:  6
n

UnexpectedAlertPresentException: Alert Text: 네이버 로그인 하신 후 이용해 주시기 바랍니다.
Message: unexpected alert open: {Alert text : 네이버 로그인 하신 후 이용해 주시기 바랍니다.}
  (Session info: chrome=89.0.4389.128)


In [ ]:
id_list=[] ; nick_list=[] ; chat_list=[]
idlst=[] ; nicklst=[] ; chatlst=[] ; DataFrame_id={} ; DataFrame_chat={}

for i in title_num:
    
    id_list.clear()
    nick_list.clear()
    chat_list.clear()
    
    time.sleep(1)
    
    #전체 웹툰 목록 중 월요일 첫 번째 웹툰으로 페이지 이동
    page=driver.find_elements_by_class_name('challengeTitle')
    page[i].click()
    
    time.sleep(0.5)
    
    #이동한 페이지 주소 읽고, 파싱하기
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    
    #작가님 닉네임 수집
    artist = soup.find_all('h2')
    artist = artist[1].find('span',{'class':'wrt_nm'}).text[8:]
    artist_list.append(artist)
    
    #작품 장르 수집
    genre=soup.find('span',{'class':'genre'}).text
    genre_list.append([genre])
    
    #최신 별점 평균 점수 수집 (최대 10화 분량)
    score = soup.find_all('strong')
    scorelist=[] ; ii=9
    while score[ii].text[0].isnumeric()==True:
        scorelist.append(float(score[ii].text))
        ii +=1
    score_list.append(sum(scorelist)/len(scorelist))
    
    
    time.sleep(0.5)
    
    length=driver.find_elements_by_class_name('title')
    if(len(length)>5):
        length=length[:5]
    
    #각 회차를 돌며 댓글 작성자 아이디 수집
    for j in range(len(length)):
        #해당 페이지의 회차 모두 가져오기
        titlenum=driver.find_elements_by_class_name('title')
        
        time.sleep(0.5)
        
        webnum=[y.text for y in titlenum]
        enterToon = driver.find_elements_by_partial_link_text(webnum[j])
        
        time.sleep(0.5)
        
        enterToon[0].click()
        
        #현재 댓글 부분 파싱이 안되는 설정이므로 src의 댓글창 페이지로 직접 이동
        #현재 URL주소 복사
        html = driver.current_url
        currentPage=html[html.find('?'):]
        
        #댓글창 페이지 URL
        URL2='https://comic.naver.com/comment/comment.nhn' + currentPage
        print('Title:',title_list[i],i,' : ',URL2)
        
        time.sleep(0.4)
        
        #댓글창 페이지 열기
        driver2 = webdriver.Chrome('chromedriver.exe')
        driver2.get(URL2)
        
        time.sleep(8)
        
        while True:
            try:
                #수집 편의를 위해 클린봇 댓글 필터링 기능 해제
                driver2.find_element_by_class_name('u_cbox_cleanbot_setbutton').click()
                time.sleep(0.4)
                driver2.find_element_by_class_name('u_cbox_layer_cleanbot_checkbox').click()
                time.sleep(0.4)
                driver2.find_element_by_class_name('u_cbox_layer_cleanbot_extrabutton').click()
                time.sleep(0.4)
                break
                
            except:
                driver2.close()
                driver2 = webdriver.Chrome('chromedriver.exe')
                driver2.get(URL2)
                print('driver2 ERROR')
                time.sleep(20)
                continue
        
        #베스트 댓글 아이디 목록 수집
        idbest = driver2.find_elements_by_class_name('u_cbox_id') ;jj=0 
        
        time.sleep(0.1)
        
        nickbest = driver2.find_elements_by_class_name('u_cbox_nick')
        
        time.sleep(0.4)
        
        chatbest = driver2.find_elements_by_class_name('u_cbox_contents') 
        
        time.sleep(1)
        
        while jj<len(idbest):
            id_list.append(idbest[jj].text)
            nick_list.append(nickbest[jj].text)
            chat_list.append(chatbest[jj].text)
            jj +=1
            
        time.sleep(0.3)
        
        #전체 댓글 보기 클릭 & 아이디 목록 수집
        driver2.find_element_by_partial_link_text('전체댓글').click()
        
        time.sleep(0.3)
        
        #최신 댓글부터 가장 나중의 댓글까지 창 이동하며 아이디 목록 수집
        num=0 ; start=0 ; error_back=0
        while True:
            try:
                while start<=0:
                    
                    time.sleep(0.4)
            
                    page=driver2.find_elements_by_class_name('u_cbox_num_page')
                    print('now page: ',int(page[num].text),' index: ',num)
            
                    time.sleep(0.1)
            
                    idd=driver2.find_elements_by_class_name('u_cbox_id')
            
                    time.sleep(0.1)
                
                    nickk=driver2.find_elements_by_class_name('u_cbox_nick')

                    time.sleep(0.1)
            
                    chat=driver2.find_elements_by_class_name('u_cbox_contents')
    
                    time.sleep(0.1)
            
                    while len(idd)>0:
                        id_list.append(nickk[0].text+idd[0].text)
                        #nick_list.append(nickk[0].text)
                        chat_list.append(chat[0].text)
                
                        idd.pop(0)
                        nickk.pop(0)
                        chat.pop(0)
                
                    time.sleep(0.1)
            
                    #페이지가 10의 배수이면 다음 페이지 클릭, 동일 과정 거침
                    try:
                        a=driver2.find_elements_by_class_name('u_cbox_next') #*****CLEAR v
                        if(int(page[num].text) % 10 == 0 and a[0].text != ''):
                            driver2.find_element_by_class_name('u_cbox_next').click()
                    
                            time.sleep(0.1)
                    
                            page=driver2.find_elements_by_class_name('u_cbox_num_page')
                    
                            time.sleep(0.1)
                    
                            idd=driver2.find_elements_by_class_name('u_cbox_id')
                    
                            time.sleep(0.1)
                        
                            nickk=driver2.find_elements_by_class_name('u_cbox_nick')
                            
                            time.sleep(0.1)
                    
                            chat=driver2.find_elements_by_class_name('u_cbox_contents')
                            a.clear()
                    
                            time.sleep(0.1)
            
                            while len(idd)>0:
                                id_list.append(nickk[0].text+idd[0].text)
                                #nick_list.append(nickk[0].text)
                                chat_list.append(chat[0].text)
                        
                                idd.pop(0)
                                nickk.pop(0)
                                chat.pop(0)
                            num=0
                            print('now page: ',int(page[num].text),' index: ',num)
            
                    except IndexError:
                        break
                
                    num +=1
            
                    time.sleep(0.4)
            
                    try:
                        page[num].click()
                
                    except IndexError:
                        break
            
                    page.clear()
   
                    idd.clear()
    
                    time.sleep(0.1)
        
                #댓글 페이지 닫기
                error_back=0
                driver2.close()
                break
        
            except:
                print('ERROR')
                error_back +=1
                if(error_back>10):
                    page[num].click()
                continue
                
        time.sleep(0.4)
        
        #페이지 뒤로 가기, 다시 만화 목록으로 
        driver.back()
        
        time.sleep(1)
        
        titlenum.clear()
        
        #time.sleep(0.5)
        
        enterToon.clear()
        
        #time.sleep(0.5)
        
        idbest.clear()
        nickbest.clear()
        chatbest.clear()
    
    #전체 정식연재 만화 창으로 이동
    time.sleep(1)
    print(title_list[i],"end")
    idlst.append(id_list.copy())
    nicklst.append(nick_list.copy())
    chatlst.append(chat_list.copy())
    
    driver.back()
    
    time.sleep(0.5)
    
    page.clear()
    
    time.sleep(0.5)
    length.clear()
    
#---------------------------------------------------크롤링 END--------------------------------------------------------------

#각 작품별로 데이터 개수가 가장 많은 작품만큼 '' (빈 값) 넣어줌
idc=idlst.copy()
max_idc=[len(z) for z in idc]

#각 작품별로 데이터 개수가 가장 많은 작품만큼 '' (빈 값) 넣어줌
for count in range(len(idc)):
    nn=max(max_idc)-len(chatlst[count])
    for plus in range(nn):
        idlst[count].append('')
        chatlst[count].append('')
        
#별점 평균 점수, 장르, 작가명 넣어줌
for count1 in range(len(idc)):
    idlst[count1].insert(0,score_list[count1])
    idlst[count1].insert(0,genre_list[count1])
    idlst[count1].insert(0,artist_list[count1])
    chatlst[count1].insert(0,score_list[count1])
    chatlst[count1].insert(0,genre_list[count1])
    chatlst[count1].insert(0,artist_list[count1])

#전체 데이터를 데이터 프레임으로 변환
total_data_id={} ; total_data_chat={} ; d=68
for diction in range(len(idc)):
    total_data_id[title_list[d]] = idlst[diction]
    total_data_chat[title_list[d]] = chatlst[diction]
    d +=1
    
Total_data_id = pd.DataFrame(total_data_id)
Total_data_chat = pd.DataFrame(total_data_chat)

#데이터 프레임을 CSV파일로 추출 내보내기(ID, CHAT 따로)
Total_data_id.to_csv('NaverWebtoon_id_tue1', mode='w')
Total_data_chat.to_csv('NaverWebtoon_tue1', mode='w')